# Introduction 🙉


The present project is a copy of Retail Sales Analysis and Rating Prediction by Brian Risk. Find it: https://www.kaggle.com/code/devraai/retail-sales-analysis-and-rating-prediction/notebook

The aim of the study project includes explore interesting trends and relationships. Also build a predictor model for customer rating.

I will try comment all the process to focus on learning process, it includes the conclusion part, i will try conclude this project with my own words.

**Table of contents**

1. [Libraries importing](#import)
1. [Data cleaning and preprocessing](#cleaning)
1. [Exploratory Data Analysis](#EDA)
1. [Predictor Model](#model)
1. [Conclusion](#conclusion)

## 📖 Libraries importing <a id='import'></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
%matplotlib inline

sns.set(style='whitegrid')


## 🧹 Data Cleaning and preprocessing <a id='cleaning'></a>

## 🔍 Exploratory Data Analysis <a id='EDA'></a>

## 📊 Predictor Model <a id='model'></a>

## 💭 Conclusion <a id='conclusion'></a>